In [1]:
from hashlib import sha256
import openpyxl
import xlrd
from ast import literal_eval

In [2]:
# Parameter publik untuk DSA
p = 1116674246425835791186763484099851270363
q = 558337123212917895593381742049925635181
g = 606189179136508625129240953541620883235

In [3]:
def Verify(outphash,A,sign):
    """ Verifikasi transaksi dalam bentuk outphash dengan address pengirim adalah A dan tandatangan sign = S1, S2
    """
    global p
    global q
    global g
    D = int(outphash, 16)
    S1 = sign[0]
    S2 = sign[1]
    V1 = (D*pow(S2,q-2,q))%q #S2^(q-2) = S2^(-1) mod q
    V2 = (S1*pow(S2,q-2,q))%q
    return ((pow(g,V1,p)*pow(A,V2,p))%p)%q == S1 #signature diverifikasi benar jikka persamaan di samping bernilai benar

In [4]:
def is_inp_there(path, inp):
    """ Mencari apakah suatu hash input terdapat pada daftar unspent transactions
    dan memberikan nomor baris beserta data pada baris tersebut
    """
    wb = xlrd.open_workbook(path)
    sheet = wb.sheet_by_index(0)

    isThere = False
    for row_num in range(sheet.nrows):
        row_data = sheet.row_values(row_num)
        if row_data[0] == inp:
            isThere = True
            break
    return isThere, row_num, row_data

In [5]:
wb = xlrd.open_workbook("awaiting.xlsx")
sheet = wb.sheet_by_index(0)

for row_number in range(sheet.nrows):
    inp = sheet.row_values(row_number)[0]
    inpidx = int(sheet.row_values(row_number)[1])
    outp = literal_eval(sheet.row_values(row_number)[2])
    sign = literal_eval(sheet.row_values(row_number)[4])
    hash0 = sha256((str(inp)+str(inpidx)+str(outp)).encode('utf8')).hexdigest()
    print("\n" + "Unspent hash to be transferred: " + inp + "\n"
          + "Index: " + str(inpidx) +"\n"
          + "Transfer to: " + str(outp) + "\n"
          + "Signature: " + str(sign) + "\n"
          + "Hash: " + hash0)
    transaction_sum = 0
    for i in range (0,len(outp)):
        transaction_sum += outp[i][1]
    isThere, row_num, row_data = is_inp_there("unspent.xlsx", inp)
    unspent_owners = literal_eval(row_data[1])
    isUnspent = isThere & (inpidx >= 0) & (inpidx < len(unspent_owners)) & (unspent_owners[inpidx] != 0)
    if isUnspent == False:
        print("Input is not verified \n")
    else:
        sender_address = unspent_owners[inpidx][0]
        if transaction_sum > unspent_owners[inpidx][1]:
            print("Transaction amount exceed unspent amount \n")
        else:
            if Verify(hash0,sender_address,sign) == False:
                print("Signature is not verified \n")
            else:
                print('Transaction is verified \n') 
                #Menghapus transaksi yang dipakai untuk input dari unspent output
                unspent_owners[inpidx] = 0
                wb = openpyxl.load_workbook('unspent.xlsx')
                ws = wb.worksheets[0]
                ws.cell(row=row_num+1, column=2).value = str(unspent_owners)
                #Menambahkan transaksi baru ke unspent output
                rows = [[hash0, str(outp)]]
                for row in rows:
                    ws.append(row)
                wb.save('unspent.xlsx')
                wb = openpyxl.load_workbook('newblock.xlsx')
                ws = wb.worksheets[0]
                rows = [[str(sender_address), str(outp), hash0]]
                for row in rows:
                    ws.append(row)
                wb.save('newblock.xlsx')


Unspent hash to be transferred: 1a428822908b32a273d294ad9a2a30e79b4a812d5f79cb98b73f4c406a1a3643
Index: 0
Transfer to: [[498063637122669310186914979135247239947, 35], [793833148500065718802366039211618163182, 15]]
Signature: [70960175482105270864450914338080137500, 3417003949243264545771660781961748864]
Hash: 3114d9838781b5d2ef9b9abaee6bc7e1531345098fcf109d7968e8710392c377
Transaction is verified 



In [6]:
#Menghapus transaksi dari awaiting list
book = openpyxl.Workbook()
sheet = book.active

book.save("awaiting.xlsx")